In [1]:
import torch
import torch.nn as nn

/home/fourierl/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
a=torch.tensor([[1,2,3,1],[4,5,6,1],[7,8,9,1]])
b=torch.tensor([[2],[3],[4]])
a*b+b

tensor([[ 4,  6,  8,  4],
        [15, 18, 21,  6],
        [32, 36, 40,  8]])

下采样

In [2]:
conv1d=nn.Conv1d(3,6,3,2,1)
input=torch.randn(16,3,34)
output=conv1d(input)
output.shape

torch.Size([16, 6, 17])

上采样

In [4]:
tconv1d=nn.ConvTranspose1d(3,6,3,2,1)
input=torch.randn(16,3,34)
output=tconv1d(input)
output.shape

torch.Size([16, 6, 67])

Conv1dBlock
仅改变通道，特征维度不发生变化

In [5]:
inp_channels=3
out_channels=6
kernel_size=3
n_groups=6
block = nn.Sequential(
        nn.Conv1d(inp_channels, out_channels, kernel_size, padding=kernel_size // 2),
        nn.GroupNorm(n_groups, out_channels),
        nn.Mish(),
        )

In [6]:
ipt=torch.randn(64,3,64)
out=block(ipt)
out.shape

torch.Size([64, 6, 64])

ConditionalResidualBlock1D

In [ ]:
in_channels=3
out_channels=6
kernel_size=3
n_groups=6
cond_dim=128
blocks = nn.ModuleList([
            block(in_channels, out_channels, kernel_size, n_groups=n_groups),
            block(out_channels, out_channels, kernel_size, n_groups=n_groups),
        ])
cond_channels = out_channels * 2
out_channels = out_channels
cond_encoder = nn.Sequential(
    nn.Mish(),
    nn.Linear(cond_dim, cond_channels),
    nn.Unflatten(-1, (-1, 1))
)

In [7]:
ipt=torch.randn(16,2,3,128,128)
fipt=ipt.flatten(end_dim=1)
fipt.shape

torch.Size([32, 3, 128, 128])

In [1]:
from transformers import BertTokenizer, BertModel
import torch

# 选择一个预训练的BERT模型
model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# 定义你的句子
sentences = ["i love you", "i eat an apple"]

# 使用tokenizer对句子进行编码
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# 获取BERT模型的输出
with torch.no_grad():
    outputs = model(**encoded_input)

# 获取[CLS]标记的向量
cls_embeddings = outputs.last_hidden_state[:, 0, :]  # shape: (batch_size, hidden_size)

# 定义一个线性层来映射到128维
linear_layer = torch.nn.Linear(768, 128)

# 将特征向量映射到128维
final_embeddings = linear_layer(cls_embeddings)

# 打印结果
print(final_embeddings)


/home/fourierl/miniconda3/envs/diff/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


OSError: Can't load tokenizer for 'bert-base-uncased'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'bert-base-uncased' is the correct path to a directory containing all relevant files for a BertTokenizer tokenizer.

In [6]:
import torch
embed=torch.nn.Embedding(5,16)
a=torch.tensor([0,1,2])
out=embed(a)
out

tensor([[-1.6093, -1.5810, -1.8347, -0.0046,  0.5608, -0.3547, -0.1353,  0.2342,
         -0.1629,  1.2530,  0.8138, -0.0986, -1.4548,  0.1548,  0.0986,  1.0534],
        [ 0.0329, -1.0724,  0.8470,  0.6794,  1.1937,  0.4995,  0.4930,  0.2056,
         -1.6767,  0.4747,  0.0064, -2.5292, -1.0586, -0.5395, -0.0628,  0.4563],
        [-0.2788, -2.7045,  0.4898, -1.0112, -0.3134,  0.6064, -1.8428, -0.1203,
         -1.1878, -0.5004,  0.9958, -0.8494, -0.5869,  0.3847,  0.7377,  0.0769]],
       grad_fn=<EmbeddingBackward0>)